In [1]:
## mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
! pip install category_encoders

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from category_encoders import TargetEncoder

from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import precision_recall_curve

import sklearn.metrics as sm

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

from sklearn.dummy import DummyRegressor

from sklearn.tree import DecisionTreeRegressor, export_graphviz

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from imblearn.under_sampling import RandomUnderSampler

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR

from xgboost import XGBRegressor

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
# Load the data

df = pd.read_csv('drive/MyDrive/Hotel-Room-Price-Estimation/Data/df_final.csv',index_col = 0)

# drop the columns that includes only one value
# and categorical variables that has too many values
df = df.drop(columns = ['israteperstay'])

# convert to some binary features to bool
df['ispromo']  = (df['ispromo']== 'Y')

df['Source']  = (df['Source']== 5)

# Use target encoding to transform the categorical variables that have too many unique values
encoder = TargetEncoder()
df[['roomtype','city','country','ratetype','propertytype']] = encoder.fit_transform(df[['roomtype',
                                                                                        'city',
                                                                                        'country',
                                                                                        'ratetype',
                                                                                        'propertytype']], 
                                                                                    df['price'])

# convert the boolean terms into integers
df = df*1

# extract the feature and target from the dataset
X_dev = df.loc[:, df.columns != 'price']
y_dev = df.loc[:, df.columns == 'price']

# train test split
X_train, X_test, y_train, y_test = train_test_split(X_dev, 
                                                    y_dev, 
                                                    test_size=0.2,  
                                                    random_state=123)

## Baseline Regressor

In [7]:
# Create a baseline regressor to examine advanced models
dummy_regr = DummyRegressor(strategy="median")

dummy_regr.fit(X_train, y_train)

train_mae_dummy = sm.mean_absolute_error(y_train, 
                                   dummy_regr.predict(X_train))

test_mae_dummy = sm.mean_absolute_error(y_test, 
                                  dummy_regr.predict(X_test))

print(f"The best training MAE is   : {round(train_mae_dummy,3)}")
print(f"The best testing MAE is    : {round(test_mae_dummy,3)}")

print(f"The training accuracy is   : {round(dummy_regr.score(X_train, y_train)*100,4)}%")
print(f"The training accuracy is   : {round(dummy_regr.score(X_test, y_test)*100,4)}%")

The best training MAE is   : 63.856
The best testing MAE is    : 63.046
The training accuracy is   : -2.404%
The training accuracy is   : -2.5309%


## Decision Tree Regressor

### Default Settings

In [ ]:
# tree_para = {'max_depth':[25,50,75,100],
#              'max_features':[15,20,25,30]}

# clf_dt = GridSearchCV(DecisionTreeRegressor(random_state = 123), 
#                    tree_para, 
#                    cv=5)

# clf_dt.fit(X_train, y_train)

# train_mae_dt = sm.mean_absolute_error(y_train, 
#                                    clf_dt.best_estimator_.predict(X_train))

# test_mae_dt = sm.mean_absolute_error(y_test, 
#                                   clf_dt.best_estimator_.predict(X_test))

# print(f"The optimal parameter is   : {clf_dt.best_params_}")
# print(f"The best training MAE is   : {round(train_mae_dt,3)}")
# print(f"The best testing MAE is    : {round(test_mae_dt,3)}")

# print(f"The training accuracy is   : {round(clf_dt.score(X_train, y_train)*100,4)}%")
# print(f"The testing accuracy is    : {round(clf_dt.score(X_test, y_test)*100,4)}%")

# The optimal parameter is   : {'max_depth': 25, 'max_features': 15}

In [ ]:
dt = DecisionTreeRegressor(random_state = 123, 
                           max_depth= 25, 
                           max_features= 15)

dt.fit(X_train, y_train)

train_mae_dt = sm.mean_absolute_error(y_train, 
                                      dt.predict(X_train))

test_mae_dt = sm.mean_absolute_error(y_test, 
                                     dt.predict(X_test))

print(f"The best training MAE is   : {round(train_mae_dt,3)}")
print(f"The best testing MAE is    : {round(test_mae_dt,3)}")

print(f"The training accuracy is   : {round(dt.score(X_train, y_train)*100,4)}%")
print(f"The testing accuracy is    : {round(dt.score(X_test, y_test)*100,4)}%")

The best training MAE is   : 12.258
The best testing MAE is    : 27.416
The training accuracy is   : 89.1485%
The testing accuracy is    : 7.488%


### Feature Selection
Consisting with the results above, we could find that there is an obvious overfitting existing in the model above (the training accuracy greatly exceeds the testing accuracy). Thus, some techniques will be implemented to avoid the overfitting.

In [ ]:
feature_importance = dt.feature_importances_

k = 5

idx = np.argpartition(feature_importance, k)

# drop the 5 least important features
X_train_simplified = X_train.drop(columns = X_train.columns[idx[:k]])

X_test_simplified = X_test.drop(columns = X_train.columns[idx[:k]])

In [ ]:
tree_para = {'max_depth':[25,50,75,100],
             'max_features':[15,20,25,30]}

clf_dt_simp = GridSearchCV(DecisionTreeRegressor(random_state = 123), 
                   tree_para, 
                   cv=5)

clf_dt_simp.fit(X_train_simplified, y_train)

train_mae_dt_simp = sm.mean_absolute_error(y_train, 
                                   clf_dt_simp.best_estimator_.predict(X_train_simplified))

test_mae_dt_simp = sm.mean_absolute_error(y_test, 
                                  clf_dt_simp.best_estimator_.predict(X_test_simplified))

print(f"The optimal parameter is   : {clf_dt_simp.best_params_}")
print(f"The best training MAE is : {round(train_mae_dt_simp,3)}")
print(f"The best testing MAE is  : {round(test_mae_dt_simp,3)}")

print(f"The training accuracy is   : {round(clf_dt_simp.score(X_train_simplified , y_train)*100,4)}%")
print(f"The testing accuracy is    : {round(clf_dt_simp.score(X_test_simplified, y_test)*100,4)}%")

# The optimal parameter is   : {'bootstrap': False, 'max_depth': 50, 'max_features': 'log2', 'min_samples_split': 5}

The optimal parameter is   : {'max_depth': 20, 'max_features': 20}
The best training MAE is : 17.109
The best testing MAE is  : 29.514
The training accuracy is   : 88.1909%
The testing accuracy is    : 1.522%


In [ ]:
dt_simp = DecisionTreeRegressor(random_state = 123, 
                                max_depth = 25, 
                                max_features= 15)

dt_simp.fit(X_train_simplified, y_train)

train_mae_dt_simp = sm.mean_absolute_error(y_train, 
                                      dt_simp.predict(X_train_simplified))

test_mae_dt_simp = sm.mean_absolute_error(y_test, 
                                     dt_simp.predict(X_test_simplified))

print(f"The best training MAE is   : {round(train_mae_dt_simp,3)}")
print(f"The best testing MAE is    : {round(test_mae_dt_simp,3)}")

print(f"The training accuracy is   : {round(dt_simp.score(X_train_simplified, y_train)*100,4)}%")
print(f"The testing accuracy is    : {round(dt_simp.score(X_test_simplified, y_test)*100,4)}%")

The best training MAE is   : 9.965
The best testing MAE is    : 26.725
The training accuracy is   : 89.5845%
The testing accuracy is    : 9.4382%


Thus, we could see that simply the feature selection could not solve the issue of overfitting. Thus, another accessible method to reduce the overfitting is to use an ensemble model.

## Random Forest Regressor

### Default Model

In [ ]:
# tree_para = {'max_depth':[25,50,75,100,125,150],
#              'max_features':['auto', 'sqrt', 'log2'],
#              'min_samples_split': [2, 5, 10],
#              'bootstrap':[True, False]
#              }

# clf_rf = GridSearchCV(RandomForestRegressor(random_state = 123), 
#                    tree_para, 
#                    cv=5)

# clf_rf.fit(X_train, y_train)

# train_mae_rf = sm.mean_absolute_error(y_train, 
#                                       clf_rf.best_estimator_.predict(X_train))

# test_mae_rf = sm.mean_absolute_error(y_test, 
#                                      clf_rf.best_estimator_.predict(X_test))

# print(f"The optimal parameter is   : {clf_rf.best_params_}")
# print(f"The best training MAE is   : {round(train_mae_rf,3)}")
# print(f"The best testing MAE is    : {round(test_mae_rf,3)}")

# print(f"The training accuracy is   : {round(clf_rf.score(X_train, y_train)*100,4)}%")
# print(f"The testing accuracy is    : {round(clf_rf.score(X_test, y_test)*100,4)}%")

# The optimal parameter is   : {'bootstrap': False, 'max_depth': 50, 'max_features': 'log2', 'min_samples_split': 5}

In [ ]:
# Use the complete features to train the random forest model
rf = RandomForestRegressor(random_state = 123, 
                           bootstrap = False, 
                           max_depth = 50, 
                           max_features = 'log2',
                           min_samples_split = 5)

rf.fit(X_train, y_train)

train_mae_rf = sm.mean_absolute_error(y_train, 
                                      rf.predict(X_train))

test_mae_rf = sm.mean_absolute_error(y_test, 
                                     rf.predict(X_test))

print(f"The best training MAE is   : {round(train_mae_rf,3)}")
print(f"The best testing MAE is    : {round(test_mae_rf,3)}")

print(f"The training accuracy is   : {round(rf.score(X_train, y_train)*100,4)}%")
print(f"The testing accuracy is    : {round(rf.score(X_test, y_test)*100,4)}%")

The best training MAE is   : 11.368
The best testing MAE is    : 22.871
The training accuracy is   : 84.9923%
The testing accuracy is    : 25.0877%


### Gradient Boosting Regressor

In [6]:
# tree_para = {'n_estimators':[25,50,75,100],
#              'max_depth':[25,50,75,100],
#              }

# clf_gbr = GridSearchCV(GradientBoostingRegressor(random_state = 123), 
#                    tree_para, 
#                    cv=5)

# clf_gbr.fit(X_train, y_train)

# train_mae_gbr = sm.mean_absolute_error(y_train, 
#                                    clf_gbr.best_estimator_.predict(X_train))

# test_mae_gbr = sm.mean_absolute_error(y_test, 
#                                   clf_gbr.best_estimator_.predict(X_test))

# print(f"The optimal parameter is   : {clf_gbr.best_params_}")
# print(f"The best training MAE is : {round(train_mae_gbr,3)}")
# print(f"The best testing MAE is  : {round(test_mae_gbr,3)}")

# print(f"The training accuracy is   : {round(clf_gbr.best_estimator_.score(X_train, y_train)*100,4)}%")
# print(f"The testing accuracy is    : {round(clf_gbr.best_estimator_.score(X_test, y_test)*100,4)}%")

# The optimal parameter is   : {'max_depth': 50, 'n_estimators': 25}

The optimal parameter is   : {'max_depth': 50, 'n_estimators': 25}
The best training MAE is : 9.255
The best testing MAE is  : 25.547
The training accuracy is   : 89.6951%
The testing accuracy is    : 16.3957%


In [ ]:
gbr = GradientBoostingRegressor(random_state = 123, 
                           max_depth =50, 
                           n_estimators=25)

gbr.fit(X_train, y_train)

train_mae_gbr = sm.mean_absolute_error(y_train, 
                                      gbr.predict(X_train))

test_mae_gbr = sm.mean_absolute_error(y_test, 
                                     gbr.predict(X_test))

print(f"The best training MAE is   : {round(train_mae_gbr,3)}")
print(f"The best testing MAE is    : {round(test_mae_gbr,3)}")

print(f"The training accuracy is   : {round(gbr.score(X_train, y_train)*100,4)}%")
print(f"The testing accuracy is    : {round(gbr.score(X_test, y_test)*100,4)}%")

In [7]:
tree_para = {'n_estimators':[25,50,75,100], 
             'max_depth':[25,50,75,100], 
             }

clf_xgb = GridSearchCV(XGBRegressor(random_state = 123,
                                verbosity = 0), 
                   tree_para, 
                   cv=5)

clf_xgb.fit(X_train, y_train)

train_mae_xgb = sm.mean_absolute_error(y_train, 
                                   clf_xgb.best_estimator_.predict(X_train))

test_mae_xgb = sm.mean_absolute_error(y_test, 
                                  clf_xgb.best_estimator_.predict(X_test))

print(f"The optimal parameter is   : {clf_xgb.best_params_}")
print(f"The best training MAE is : {round(train_mae_xgb,3)}")
print(f"The best testing MAE is  : {round(test_mae_xgb,3)}")

print(f"The training accuracy is   : {round(clf_xgb.best_estimator_.score(X_train, y_train)*100,4)}%")
print(f"The testing accuracy is    : {round(clf_xgb.best_estimator_.score(X_test, y_test)*100,4)}%")

# The optimal parameter is   : {'max_depth': 25, 'n_estimators': 25}

The optimal parameter is   : {'max_depth': 25, 'n_estimators': 25}
The best training MAE is : 16.486
The best testing MAE is  : 25.417
The training accuracy is   : 86.6686%
The testing accuracy is    : 25.3365%


In [ ]:
xgb = XGBRegressor(random_state = 123,
                   verbosity = 0,
                   max_depth = 25, 
                   n_estimators=25
                   )

xgb.fit(X_train, y_train)

train_mae_xgb = sm.mean_absolute_error(y_train, 
                                   xgb.predict(X_train))

test_mae_xgb = sm.mean_absolute_error(y_test, 
                                  xgb.predict(X_test))

print(f"The best training MAE is : {round(train_mae_xgb,3)}")
print(f"The best testing MAE is  : {round(test_mae_xgb,3)}")

print(f"The training accuracy is   : {round(xgb.score(X_train, y_train)*100,4)}%")
print(f"The testing accuracy is    : {round(xgb.score(X_test, y_test)*100,4)}%")